In [27]:
import ipywidgets
import ipyleaflet
from IPython.display import display
import requests
from collections import defaultdict

In [39]:
center_of_dc = (38.898569, -77.031008)



lines = {
    'Green': {'code': 'GR', 'color': '#00FF00', 'stations': [], 'incidents': []},
    'Red': {'code': 'RD', 'color': '#BE1337', 'stations': [], 'incidents': []},
    'Orange': {'code': 'OR', 'color': '#DA8707', 'stations': [], 'incidents': []},
    'Yellow': {'code': 'YL', 'color': '#F5D415', 'stations': [], 'incidents': []},
    'Blue': {'code': 'BL', 'color': '#0795D3', 'stations': [], 'incidents': []},
    'Silver': {'code': 'SV', 'color': '#A2A4A1', 'stations': [], 'incidents': []},
}

code2line = dict((v['code'], v) for k, v in lines.items())
station2code = defaultdict(list)

demo_key = "6b700f7ea9db408e9745c207da7ca827"
incidents_url = 'https://api.wmata.com/Incidents.svc/json/ElevatorIncidents'

hdrs = {'api_key': demo_key}
incidents = requests.get(incidents_url, headers=hdrs).json()

stations_url = 'https://api.wmata.com/Rail.svc/json/jStations'
stations = requests.get(stations_url, headers=hdrs).json()

for s in stations['Stations']:
    for field in ['LineCode1', 'LineCode2', 'LineCode3', 'LineCode4']:
        if s[field]:
            lc = code2line[s[field]]
            code2line[s[field]]['stations'].append(s)
            station2code[s['Code']].append(s[field])

for i in incidents['ElevatorIncidents']:
    sc = i['StationCode']
    for lc in station2code[sc]:
        code2line[lc]['incidents'].append(i)
            
            
map_ = None

def get_line(Line):
    global map_
    if map_:
        map_.clear_layers()
        map_.close()
    map_ = ipyleaflet.Map(default_tiles=ipyleaflet.TileLayer(opacity=1.0),
                     center=center_of_dc, zoom=13)
    
    polyline = ipyleaflet.Polyline(locations=[(x['Lat'], x['Lon']) for x in Line['stations']])
    polyline.fill_color = Line['color']
    polyline.fill_opacity = 1.0
    
    map_.add_layer(polyline)
    
    for i in Line['incidents']:
        marker = ipyleaflet.CircleMarker(location=center_of_dc, radius=5, weight=5,
                                        color=Line['color'], opacity=1.0, fill_opacity=0.5,
                                        fill_color=Line['color'])    
        map_.add_layer(marker)
    return map_

# mark = ipyleaflet.CircleMarker(location=(), radius=int, weight=int, color='#FFF', opacity=1.0, fill_opacity=0.5,
# fill_color='#FFF')
# map_.add_layer(mark)

choice_widget = ipywidgets.interactive(get_line, Line=lines)
display(choice_widget)